In [7]:
from splinter import Browser
from bs4 import BeautifulSoup 
from webdriver_manager.chrome import ChromeDriverManager
import pymongo
from flask import Flask, render_template
import pandas as pd
import requests

In [8]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\calla\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [9]:
url = 'https://redplanetscience.com/'
browser.visit(url)
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [12]:
# get the title
news_title=soup.find_all('div', class_='content_title')[0].text

news_title 


'Mars Scientists Investigate Ancient Life in Australia'

In [13]:
#get first paragraph
news_paragraph = soup.find_all('div', class_='article_teaser_body')[0].text
news_paragraph


"Teams with NASA's Mars 2020 and ESA's ExoMars practiced hunting for fossilized microbial life in the Australian Outback in preparation for their Red Planet missions. "

In [19]:
# get the featured space image 
jpl_url = "https://spaceimages-mars.com/" 
browser.visit(jpl_url)



In [33]:
html=browser.html
soup=BeautifulSoup(html,"html.parser")

image_url = [i.get("src") for i in soup.find_all("img", class_="headerimage fade-in")]
image_url[0]


'image/featured/mars1.jpg'

In [35]:
featured_image_url = jpl_url + image_url[0]
featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [21]:
### get the mars fact tables
mars_facts_url = "https://galaxyfacts-mars.com/"

In [23]:
tables = pd.read_html(mars_facts_url)

len(tables)

2

In [24]:
tables[0]

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [25]:
tables[1]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [61]:
mars_facts_table = tables[0]
mars_facts_table

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [62]:
mars_facts_table.rename(columns={0 : 'Description',1 : 'Mars',2: 'Earth'}, inplace=True)
mars_facts_table = mars_facts_table.set_index('Description')
mars_facts_table

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [65]:
mars_facts = mars_facts_table.to_html()
mars_facts.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Description</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Mars - Earth Comparison</th>      <td>Mars</td>      <td>Earth</td>    </tr>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [66]:
mars_facts_table.to_html('table.html')

In [39]:
#Mars hemispheres information
h_url = "https://marshemispheres.com/"
hemisphere_url = requests.get(h_url)

In [59]:
soup = BeautifulSoup(hemisphere_url.content)

hemisphere_items = soup.find_all("div", class_="item")

In [60]:
hemisphere_htmls = []
for x in hemisphere_items:
    for a in x.find_all('a', href=True):
        if a.text:
            hemisphere_htmls.append(a['href'])
            
hem_htmls = [h_url + x for x in hemisphere_htmls]
hem_htmls 

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [53]:
def hemisphere_images(url):
    response = requests.get(url)
    soup = bs(response.text, "html.parser")
    image = soup.find_all("img", class_="wide-image")
    hem_images = [x.get("src") for x in image]
    return "https://marshemispheres.com/" + hem_images[0]
    
img_url = [hemisphere_images(x) for x in hem_htmls]
img_url

['https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [54]:
def hem_title(url):
    response = requests.get(url)
    soup = bs(response.text, "html.parser")
    title = soup.find_all("h2", class_="title")
    title_info = [y.text for y in title]
    return title_info[0]
    
title = [hem_title(y) for y in hem_htmls]
title

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [55]:
list_hems = list(zip(title, img_url))



In [58]:
def dict_list(keys, list_of_hems):
    hems_dict = [dict(zip(keys, values)) for values in list_of_hems]
    return hems_dict

keys = ("title", "img_url")
hemisphere_image_urls = dict_list(keys, list_hems)
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

You can also save the table directly to a file.